<a href="https://colab.research.google.com/github/onlyreynaldo/python-clustering-data/blob/main/Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# **Carregando os Dados**

## Tabela de Pedidos

In [13]:
df_orders = pd.read_csv('/content/drive/MyDrive/datasets/olist/olist_orders_dataset.csv', parse_dates=['order_approved_at'])
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


# Tabela de Item-Pedidos

In [14]:
df_order_items = pd.read_csv('/content/drive/MyDrive/datasets/olist/olist_order_items_dataset.csv')
df_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


# Tabela de Cadastro de Sellers

In [15]:
df_sellers = pd.read_csv('/content/drive/MyDrive/datasets/olist/olist_sellers_dataset.csv')
df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


# Criando a ABT de Classificação

In [17]:
# criando histórico da abt de trino
df_historico_abt_train = (
    df_order_items
    .merge(df_orders, on='order_id', how='left')
    .query('order_status == "delivered"')
    .query('order_approved_at >= "2017-01-01" & order_approved_at < "2018-07-01"')
    .merge(df_sellers, on='seller_id', how='left')
)
df_historico_abt_train.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,37564,borda da mata,MG
3,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,87900,loanda,PR
4,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27,21.9,12.69,816cbea969fe5b689b39cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-17 03:55:27,2017-05-17 11:05:55,2017-05-22 13:44:35,2017-06-06 00:00:00,14091,ribeirao preto,SP


In [19]:
# criando as features
df_features_train = (
    df_historico_abt_train
    .query('order_approved_at < "2018-01-01"')
    .groupby('seller_id')
    .agg(uf = ('seller_state', 'first'),
         tot_orders_12m = ('order_id', 'nunique'),
         tot_items_12m = ('product_id', 'count'),
         tot_items_dist_12m = ('product_id', 'nunique'),
         receita_12m = ('price', 'sum'),
         data_ult_vnd = ('order_approved_at', 'max'))
    .reset_index()
    .assign(data_ref = pd.to_datetime('2018-01-01 00:00:00'))
    .assign(recencia = lambda df: (df['data_ref'] - df['data_ult_vnd']).dt.days)
)
df_features_train.head()

,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,data_ult_vnd,data_ref,recencia
0,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,2017-10-18 23:56:20,2018-01-01,74
1,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2017-12-29 02:09:49,2018-01-01,2
2,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2017-12-29 02:10:04,2018-01-01,2
3,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,2017-12-15 07:11:03,2018-01-01,16
4,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,2017-12-23 21:49:44,2018-01-01,8


In [12]:
# df_orders = pd.read_csv('/content/drive/MyDrive/datasets/olist/olist_orders_dataset.csv', parse_dates=['order_approved_at'])
# Declarar nuevamente para hacer la operacion de la resta de los dias. Recordar que una vez hecho esto correr todo de nuevo.

In [20]:
df_features_train.sort_values('recencia', ascending=False).head()

,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,data_ult_vnd,data_ref,recencia
1342,c9a06ece156bb057372c68718ec8909b,DF,9,9,1,3420.00,2017-01-15 18:05:14,2018-01-01,350
456,436bf27f2f18474fc6047702e9f8a866,SP,1,1,1,29.90,2017-01-18 13:05:15,2018-01-01,347
1453,dd264199fc8b687ad029de7de6d760e6,PR,1,1,1,87.90,2017-01-19 13:30:21,2018-01-01,346
1589,f235413f677777a0738ff215d0184901,SP,2,2,1,1191.98,2017-01-23 19:45:12,2018-01-01,342
880,822b63912576852aea9a8436d72317b7,SP,3,4,2,615.96,2017-01-24 12:25:27,2018-01-01,341


In [21]:
# criando o target
(
    df_historico_abt_train
    .query('order_approved_at >= "2018-01-01" & order_approved_at < "2018-07-01"')
    .filter(['seller_id'])
    .drop_duplicates()
)

,seller_id
2,5b51032eddd242adc84c38acab88f23d
6,a416b6a846a11724393025641d4edd5e
7,cc419e0650a3c5ba77189a1882b7556a
8,1f50f920176fa81dab994f9023523100
10,fcb5ace8bcc92f75707dc0f01a27d269
...,...
93845,686886775c8f6b1191c7b953fb52708c
94206,83e197e95a1bbabc8c75e883ed016c47
95493,3e35a8bb43569389d3cebef0ce820f69
95641,4391ea5c5990502176a1dcda44a2373d


In [22]:
# criando o target
(
    df_historico_abt_train
    .query('order_approved_at >= "2018-01-01" & order_approved_at < "2018-07-01"')
    .filter(['seller_id'])
    .drop_duplicates()
    .query('seller_id == "c9a06ece156bb057372c68718ec8909b"')
)

,seller_id


In [23]:
# criando o target
df_target_train = (
    df_historico_abt_train
    .query('order_approved_at >= "2018-01-01" & order_approved_at < "2018-07-01"')
    .filter(['seller_id'])
    .drop_duplicates()
)
df_target_train.head()

,seller_id
2,5b51032eddd242adc84c38acab88f23d
6,a416b6a846a11724393025641d4edd5e
7,cc419e0650a3c5ba77189a1882b7556a
8,1f50f920176fa81dab994f9023523100
10,fcb5ace8bcc92f75707dc0f01a27d269


In [24]:
# criando a abt de fato
(
    df_features_train
    .merge(df_target_train, on='seller_id', how='left', indicator=True)
).head()

,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,data_ult_vnd,data_ref,recencia,_merge
0,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,2017-10-18 23:56:20,2018-01-01,74,left_only
1,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2017-12-29 02:09:49,2018-01-01,2,both
2,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2017-12-29 02:10:04,2018-01-01,2,both
3,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,2017-12-15 07:11:03,2018-01-01,16,left_only
4,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,2017-12-23 21:49:44,2018-01-01,8,both


In [25]:
# criando a abt de fato
(
    df_features_train
    .merge(df_target_train, on='seller_id', how='left', indicator=True)
    .assign(churn_6m = lambda df: np.where(df['_merge'] == "left_only", 1, 0))
).head()

,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,data_ult_vnd,data_ref,recencia,_merge,churn_6m
0,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,2017-10-18 23:56:20,2018-01-01,74,left_only,1
1,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2017-12-29 02:09:49,2018-01-01,2,both,0
2,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2017-12-29 02:10:04,2018-01-01,2,both,0
3,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,2017-12-15 07:11:03,2018-01-01,16,left_only,1
4,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,2017-12-23 21:49:44,2018-01-01,8,both,0


In [27]:
# criando a abt de fato
df_abt_churn_train = (
    df_features_train
    .merge(df_target_train, on='seller_id', how='left', indicator=True)
    # left_only = churn (1), both = não churn (0)
    .assign(churn_6m = lambda df: np.where(df['_merge'] == "left_only", 1, 0))
    .filter(['data_ref', 
             'seller_id',
             'uf',
             'tot_orders_12m',
             'tot_items_12m',
             'tot_items_dist_12m',
             'receita_12m',
             'recencia',
             'churn_6m'])
)
df_abt_churn_train.head()

,data_ref,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,churn_6m
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,74,1
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2,0
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2,0
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,16,1
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,8,0


# Criando a ABT Out of Time (Validação ou Teste)

In [28]:
# criando histórico da abt de treino
df_historico_abt_oot = (
    df_order_items
    .merge(df_orders, on='order_id', how='left')
    .query('order_status == "delivered"')
    .query('order_approved_at >= "2017-02-01" & order_approved_at < "2018-08-01"')
    .merge(df_sellers, on='seller_id', how='left')
)
df_historico_abt_train.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,37564,borda da mata,MG
3,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,87900,loanda,PR
4,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27,21.9,12.69,816cbea969fe5b689b39cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-17 03:55:27,2017-05-17 11:05:55,2017-05-22 13:44:35,2017-06-06 00:00:00,14091,ribeirao preto,SP


In [29]:
df_historico_abt_oot.agg({'order_approved_at': ['min', 'max']})

,order_approved_at
min,2017-02-01 02:35:18
max,2018-07-31 23:55:18


In [30]:
# criando as features
df_features_oot = (
    df_historico_abt_oot
    .query('order_approved_at < "2018-02-01"')
    .groupby('seller_id')
    .agg(uf = ('seller_state', 'first'),
         tot_orders_12m = ('order_id', 'nunique'),
         tot_items_12m = ('product_id', 'count'),
         tot_items_dist_12m = ('product_id', 'nunique'),
         receita_12m = ('price', 'sum'),
         data_ult_vnd = ('order_approved_at', 'max'))
    .reset_index()   
    .assign(data_ref = pd.to_datetime('2018-02-01 00:00:00'))
    .assign(recencia = lambda df: (df['data_ref'] - df['data_ult_vnd']).dt.days)
)
df_features_oot.head()

,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,data_ult_vnd,data_ref,recencia
0,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,2017-10-18 23:56:20,2018-02-01,105
1,001cca7ae9ae17fb1caed9dfb1094831,ES,179,215,9,22189.23,2018-01-29 21:13:27,2018-02-01,2
2,002100f778ceb8431b7a1020ff7ab48f,SP,41,45,18,859.50,2018-01-15 17:18:23,2018-02-01,16
3,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,2017-12-15 07:11:03,2018-02-01,47
4,004c9cd9d87a3c30c522c48c4fc07416,SP,132,143,77,16619.10,2018-01-31 13:32:35,2018-02-01,0


In [31]:
# criando o target
df_target_oot = (
    df_historico_abt_oot
    .query('order_approved_at >= "2018-02-01" & order_approved_at < "2018-08-01"')
    .filter(['seller_id'])
    .drop_duplicates()
)
df_target_oot.head()

,seller_id
6,5996cddab893a4652a15592fb58ab8db
7,a416b6a846a11724393025641d4edd5e
8,ba143b05f0110f0dc71ad71b4466ce92
9,cc419e0650a3c5ba77189a1882b7556a
10,8602a61d680a10a82cceeeda0d99ea3d


In [32]:
# criando a abt out of time
df_abt_churn_oot = (
    df_features_oot
    .merge(df_target_oot, on='seller_id', how='left', indicator=True)
    # left_only = churn (1), both = não churn (0)
    .assign(churn_6m = lambda df: np.where(df['_merge'] == "left_only", 1, 0))
    .filter(['data_ref', 
             'seller_id',
             'uf',
             'tot_orders_12m',
             'tot_items_12m',
             'tot_items_dist_12m',
             'receita_12m',
             'recencia',
             'churn_6m'])
)
df_abt_churn_oot.head()

,data_ref,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,churn_6m
0,2018-02-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,105,1
1,2018-02-01,001cca7ae9ae17fb1caed9dfb1094831,ES,179,215,9,22189.23,2,0
2,2018-02-01,002100f778ceb8431b7a1020ff7ab48f,SP,41,45,18,859.50,16,0
3,2018-02-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,47,1
4,2018-02-01,004c9cd9d87a3c30c522c48c4fc07416,SP,132,143,77,16619.10,0,0


# Verificando as Distribuições da Variável Target

In [33]:
df_abt_churn_train['churn_6m'].value_counts(normalize=True)

0    0.623669
1    0.376331
Name: churn_6m, dtype: float64

In [34]:
df_abt_churn_oot['churn_6m'].value_counts(normalize=True)

0    0.614404
1    0.385596
Name: churn_6m, dtype: float64

In [35]:
df_abt_churn_oot['churn_6m'].value_counts()

0    1109
1     696
Name: churn_6m, dtype: int64

In [36]:
df_abt_churn_train['churn_6m'].value_counts()

0    1054
1     636
Name: churn_6m, dtype: int64

# Salvando as ABTs

In [39]:
import joblib

joblib.dump(df_abt_churn_train, '/content/drive/MyDrive/datasets/olist/abt_classificacao_churn_train.pkl')

['/content/drive/MyDrive/datasets/olist/abt_classificacao_churn_train.pkl']

In [41]:
joblib.load('/content/drive/MyDrive/datasets/olist/abt_classificacao_churn_train.pkl')

,data_ref,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,churn_6m
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,74,1
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2,0
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2,0
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,16,1
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,8,0
...,...,...,...,...,...,...,...,...,...
1685,2018-01-01,ff82e8873fba613f2261a9acc896fd84,MG,1,1,1,14.90,25,0
1686,2018-01-01,ffdd9f82b9a447f6f8d4b91554cc7dd3,PR,8,9,7,1254.10,34,0
1687,2018-01-01,ffeee66ac5d5a62fe688b9d26f83f534,SP,8,8,2,1099.92,8,0
1688,2018-01-01,fffd5413c0700ac820c7069d66d98c89,SP,19,20,15,3374.30,6,0


In [37]:
df_abt_churn_train.to_csv('/content/drive/MyDrive/datasets/olist/abt_classificacao_churn_train.csv', index=False)

In [38]:
df_abt_churn_oot.to_csv('/content/drive/MyDrive/datasets/olist/abt_classificacao_churn_oot.csv', index=False)